In [ ]:
# The project aims to scrape the data on landfills in Georgia, from the interactive map embedded on the webpage of 'Waste.Gov.Ge'
# The webpage address: http://waste.gov.ge/ka/?page_id=3216

In [ ]:
# LIBRARIES

In [1]:
# Read libraries
from urllib.request import urlopen
import re
import json
import pandas as pd
import numpy as np

In [ ]:
# HTML

In [2]:
# Open the URL ('urlopen()'), read the HTML content ('read()') and assign encoding ('decode()')
url = "http://waste.gov.ge/ka?page_id=3216"
html_content = urlopen(url).read().decode('utf-8')  # Assuming the content is encoded in UTF-8

In [ ]:
# Check the content in HTML
#print(html_content)

In [3]:
# Extract Title From HTML With String Methods

title_index = html_content.find("<title>")
#title_index
start_index = title_index + len("<title>")
#start_index
end_index = html_content.find("</title>")
#end_index

title = html_content[start_index:end_index]
title

'არსებული ნაგავსაყრელები - Waste.Gov.Ge - LTD Solid Waste Management Company of Georgia'

In [4]:
# In the HTML, the content that is needed, is stored within 'text/javascript', function 'fusion_maps', addresses: [{}].
# Regular expression is used to extract only the needed part from the whole html.
# The addresses part contains the JSON data.

# Define the regex pattern to capture the addresses part
pattern = r'addresses:\s*(\[.*?\])'

# Find all matches of the pattern in the HTML content
matches = re.findall(pattern, html_content, re.DOTALL)

# Check if there are any matches
if matches:
    # Convert the JSON string to a Python list of dictionaries
    data = json.loads(matches[0])

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame as a CSV file with UTF-8 encoding
    #df.to_csv('path/data.csv', index=False, encoding='utf-8-sig')
    print("DataFrame saved as data.csv")
else:
    print("No matches found.")

DataFrame saved as data.csv


In [ ]:
# Check the results
#df

In [ ]:
# DATA CLEANING

In [5]:
# In the dataframe, the column 'infobox_content' represents list of key-value pairs. Further steps are required to extract key-values

# Remove '</p><p>' and then split by '<br />'
infobox_split = df['infobox_content'].str.replace('</p>\n<p>', '').str.split('<br />', expand=True)


# Create a dictionary to store key-value pairs for each row
infobox_dict = {}

# Iterate over each row
for index, row in infobox_split.iterrows():
    # Initialize a dictionary to store key-value pairs for the current row
    row_dict = {}
    # Iterate over each pair in the current row
    for pair in row:
        if pair:
            # Split the pair by colon (':')
            pair_parts = pair.split(':', 1) # 1 (max_split) means that it splits by first ':' 
            # Check if the split result has two parts (key and value)
            if len(pair_parts) == 2:
                # Extract key-value pairs
                key, value = pair_parts
                # Store key-value pairs in the dictionary
                row_dict[key.strip()] = value.strip()
            elif len(pair_parts) == 1:
                # If only one part is found, assume it's a value without a key
                # Store the value with a placeholder key
                row_dict[f'Value_{len(row_dict)+1}'] = pair_parts[0].strip()
    # Store the dictionary for the current row in the main dictionary
    infobox_dict[index] = row_dict

# Convert the dictionary into a DataFrame
infobox_df = pd.DataFrame.from_dict(infobox_dict, orient='index')

# Reset the index to ensure correct alignment
infobox_df.reset_index(drop=True, inplace=True)

# Concatenate the infobox_df with the original DataFrame
df = pd.concat([df, infobox_df], axis=1)

# To drop the original 'infobox_content' column | but in this case it will be kept
#df.drop('infobox_content', axis=1, inplace=True)

In [ ]:
# Check the results
#df

In [6]:
# Check the types of columns
print(df.dtypes)

address                               object
infobox_content                       object
marker                                object
coordinates                             bool
cache                                   bool
latitude                              object
longitude                             object
Value_1                               object
სტატუსი                               object
მისამართი                             object
ნაგავსაყრელის ტერიტორია               object
Value_3                               object
სამუშაო საათები                       object
Value_6                               object
ნარჩენების საშუალო რაოდენობა თვეში    object
dtype: object


In [7]:
# Get column names
df.columns

Index(['address', 'infobox_content', 'marker', 'coordinates', 'cache',
       'latitude', 'longitude', 'Value_1', 'სტატუსი', 'მისამართი',
       'ნაგავსაყრელის ტერიტორია', 'Value_3', 'სამუშაო საათები', 'Value_6',
       'ნარჩენების საშუალო რაოდენობა თვეში'],
      dtype='object')

In [8]:
# Replace column names for some occurencies
df = df.rename(columns={'address': 'place', 
                            'Value_1': 'სახელი', 
                            'ნაგავსაყრელის ტერიტორია': 'ნაგავსაყრელის ტერიტორია (კვ.მ.)', 
                            'Value_3': 'ემსახურება', 
                            'Value_6': 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)'})

In [ ]:
#df

In [9]:
# Format column values

# 1. Column 'ნაგავსაყრელის ტერიტორია (კვ.მ.)'

# Remove ' m2' suffix from the values in the 'ნაგავსაყრელის ტერიტორია (კვ.მ.)' column
df['ნაგავსაყრელის ტერიტორია (კვ.მ.)'] = df['ნაგავსაყრელის ტერიტორია (კვ.მ.)'].str.replace(' m2', '')
# Convert the 'ნაგავსაყრელის ტერიტორია (კვ.მ.)' column to float dtype
df['ნაგავსაყრელის ტერიტორია (კვ.მ.)'] = df['ნაგავსაყრელის ტერიტორია (კვ.მ.)'].str.replace(' ', '').astype(float)


# 2. Column 'ემსახურება'

# Remove 'ემსახურება ' suffix from the values in the 'ემსახურება' column
df['ემსახურება'] = df['ემსახურება'].str.replace('ემსახურება ', '').str.replace(' და ', ', ')


# 3. Column 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)'
# Columns 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)' & 'ნარჩენების საშუალო რაოდენობა თვეში' 
# have assigned the values of similar variable but part of them are stored in the first column and part of them in second.
# It's needed to combine both of them and store them under one column ('ნარჩენების საშუალო რაოდენობა თვეში (მ3)')

# Combine the values from both columns into a new column and then drop the original columns.
# Combine values from both columns
df['combined'] = df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'].fillna('') + df['ნარჩენების საშუალო რაოდენობა თვეში'].fillna('')
# Drop original columns
df.drop(columns=['ნარჩენების საშუალო რაოდენობა თვეში (მ3)', 'ნარჩენების საშუალო რაოდენობა თვეში'], inplace=True)
# Rename new column
df = df.rename(columns={'combined': 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)'})


# 4. Column 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)'
# Now values need to format, so only numbers would be left and then to convert string as float
# Remove ' m3' suffix from the values in the 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)' column
df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'] = df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'].str.replace(' m3', '')
# Remove extra text ('ნარჩენების საშუალო რაოდენობა თვეში - ') from the values in the 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)' column
df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'] = df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'].str.replace('ნარჩენების საშუალო რაოდენობა თვეში - ', '')
df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'] = df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'].str.replace('ნარჩენების საშუალო რაოდენობა თვეში -', '')
# Replace empty strings with NaN
df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'] = df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'].replace('', np.nan)
# Convert the 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)' column from string to float
df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'] = df['ნარჩენების საშუალო რაოდენობა თვეში (მ3)'].astype(float)


In [ ]:
#df

In [ ]:
# Format coordinates

In [10]:
# Step 1: Check Data Consistency: ensure that all latitude values fall within the valid range of -90 to +90 degrees, and all longitude values fall within the valid range of -180 to +180 degrees. Values outside these ranges are invalid and should be flagged for further investigation.
valid_lat_range = (-90, 90)
valid_lon_range = (-180, 180)

valid_lat = (df['latitude'].astype(float) >= valid_lat_range[0]) & (df['latitude'].astype(float) <= valid_lat_range[1])
valid_lon = (df['longitude'].astype(float) >= valid_lon_range[0]) & (df['longitude'].astype(float) <= valid_lon_range[1])

In [11]:
#valid_lat
#valid_lon

In [12]:
# Step 2: Handle Missing Values: check for any missing or null values in the latitude and longitude columns. Decide on an appropriate strategy to handle these missing values, such as imputation or removal.
df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Step 3: Remove Special Characters (if any)
# Remove Special Characters: Remove any non-numeric characters or special symbols from the latitude and longitude values, such as degree symbols or directional indicators (e.g., N, S, E, W)

In [13]:
# Step 4: Convert to Float: after cleaning the data, convert the latitude and longitude columns to float data type
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [14]:
# Check the types of columns
print(df.dtypes)

place                                       object
infobox_content                             object
marker                                      object
coordinates                                   bool
cache                                         bool
latitude                                   float64
longitude                                  float64
სახელი                                      object
სტატუსი                                     object
მისამართი                                   object
ნაგავსაყრელის ტერიტორია (კვ.მ.)            float64
ემსახურება                                  object
სამუშაო საათები                             object
ნარჩენების საშუალო რაოდენობა თვეში (მ3)    float64
dtype: object


In [15]:
# Check the results
#df

In [16]:
# Convert the English labels of places into Georgian and store in a new column 'დასახლება'

# Define a custom mapping dictionary for municipality names
places_mapping = {
    "Chiatura": "ჭიათურა",
    "Sachkhere": "საჩხერე",
    
    "Tkibuli": "ტყიბული",
    "Samtredia": "სამტრედია",
    "Terjola": "თერჯოლა",
    "Zestafoni": "ზესტაფონი",
    "Kharagauli": "ხარაგაული",
    "Khoni": "ხონი",
    "Kutaisi": "ქუთაისი",
    "Sagarejo": "საგარეჯო",
    "Gurjaani": "გურჯაანი",
    "Dedoplistskaro": "დედოფლისწყარო",
    "Kvareli": "ყვარელი",
    "Telavi": "თელავი",
    "Tsnori": "წნორი",
    "Lagodekhi": "ლაგოდეხი",
    "Martvili": "მარტვილი",
    "Chkhorotsku": "ჩხოროწყუ",
    "Senaki": "სენაკი",
    "Abasha": "აბაშა",
    "Zugdidi": "ზუგდიდი",
    "Tsalenjikha": "წალენჯიხა",
    "Khobi": "ხობი",
    "Poti": "ფოთი",
    "Chokhatauri": "ჩოხატაური",
    "Lanchkhuti": "ლანჩხუთი",
    "Ozurgeti": "ოზურგეთი",
    "Ureki": "ურეკი",
    "Khashuri": "ხაშური",
    "Kareli": "ქარელი",
    "Agara": "აგარა",
    "Gori": "გორი",
    "Kaspi": "კასპი",
    "Tetri": "თეთრი",
    "Tsalka": "წალკა",
    "Dmanisi": "დმანისი",
    "Bolnisi": "ბოლნისი",
    "Chivchavi": "ჭივჭავი",
    "Marneuli": "მარნეული",
    "Aspindza": "ასპინძა",
    "Bakuriani": "ბაკურიანი",
    "Akhaltsikhe": "ახალციხე",
    "Ninotsminda": "ნინოწმინდა",
    "Borjomi": "ბორჯომი",
    "Tsageri": "ცაგერი",
    "Oni": "ონი",
    "Ambrolauri": "ამბროლაური",
    "Tianeti": "თიანეთი",
    "Qsani": "ქსანი",
    "Sioni": "სიონი",
    "Dusheti": "დუშეთი",
    "Stepantsminda": "სტეფანწმინდა"
    # Add more mappings as needed
}


# Function to convert English names to Georgian
def convert_to_georgian(place):
    return places_mapping.get(place, place)

# Apply the conversion function to create the 'დასახლება' field
df['დასახლება'] = df['place'].apply(convert_to_georgian)

In [ ]:
# Check the results
#df

In [17]:
# Get the column names
df.columns

Index(['place', 'infobox_content', 'marker', 'coordinates', 'cache',
       'latitude', 'longitude', 'სახელი', 'სტატუსი', 'მისამართი',
       'ნაგავსაყრელის ტერიტორია (კვ.მ.)', 'ემსახურება', 'სამუშაო საათები',
       'ნარჩენების საშუალო რაოდენობა თვეში (მ3)', 'დასახლება'],
      dtype='object')

In [18]:
# Order columns in a desired order

# Define column order
desired_order = ['place', 'infobox_content', 'marker', 'coordinates', 'cache', 
                 'latitude', 'longitude', 'სახელი', 'დასახლება', 'სტატუსი', 'მისამართი', 
                 'ნაგავსაყრელის ტერიტორია (კვ.მ.)', 'ემსახურება', 'სამუშაო საათები', 
                 'ნარჩენების საშუალო რაოდენობა თვეში (მ3)']

# Reorder the columns
df = df.reindex(columns=desired_order)

In [18]:
# Check the results
df.head(2)

,place,infobox_content,marker,coordinates,cache,latitude,longitude,სახელი,დასახლება,სტატუსი,მისამართი,ნაგავსაყრელის ტერიტორია (კვ.მ.),ემსახურება,სამუშაო საათები,ნარჩენების საშუალო რაოდენობა თვეში (მ3)
0,Chiatura,ჭიათურის ნაგავსაყრელი<br />\nსტატუსი : დახურულ...,http://waste.gov.ge/ka/wp-content/uploads/2018...,False,True,42.283333,43.283333,ჭიათურის ნაგავსაყრელი,ჭიათურა,დახურული,"ჭიათურა,სოფელი რგანი",7561.0,NaN,NaN,NaN
1,Sachkhere,</p>\n<p>საჩხერის ნაგავსაყრელი<br />\nსტატუსი ...,http://waste.gov.ge/ka/wp-content/uploads/2018...,False,True,42.345278,43.419444,საჩხერის ნაგავსაყრელი,საჩხერე,მოქმედი,"საჩხერე, სოფელი სარეკი",15980.0,"საჩხერეს, ჭიათურას",24/7,1800.0


In [19]:
# Save the csv file
#df.to_csv('/path/landfills_georgia_24.csv', index=False, encoding='utf-8-sig')